In [42]:
import numpy as np
import os
import json

In [16]:
eval_conditions = [182, 200, 229, 414, 441, 788, 93, 629, 56]

In [43]:
condition_db_file = "/shares/bulk/oagba/data/definitions/condition_db.json"
with open(condition_db_file) as fp:
    condition_db = json.load(fp)
condition_codes = sorted(condition_db.keys())

In [49]:
condition_db[condition_codes[762]]

'Phimosis'

In [5]:
nb_base_dir = "/home/oagba/bulk/data/output_basic_15k/symptoms/csv/parsed/learning/nb"
nb_top5_file = os.path.join(nb_base_dir, "naive_bayes_top_5_pred.joblib")
nb_ntop5_file = os.path.join(nb_base_dir, "naive_bayes_top_n5_pred.joblib")

In [6]:
import joblib

In [17]:
from collections import Counter

In [7]:
nb_top5_data = joblib.load(nb_top5_file)
nb_ntop5_data = joblib.load(nb_ntop5_file)

In [9]:
nb_top5_acc = nb_top5_data['acc']
nb_top5_label = nb_top5_data['labels']

In [63]:
def extract(cond, acc, label):
    summarized = {}
    ind = label == cond
    top_5_data = acc[ind, :]
    flatten = top_5_data.reshape(-1, )
    cnt = Counter(flatten[flatten != cond].tolist())
    
    return {condition_db[condition_codes[item[0]]]: item[1] for item in cnt.most_common(5)}

In [64]:
nb_summarized = {}

In [65]:
for cond in eval_conditions:
    cond_name = condition_db[condition_codes[cond]]
    nb_summarized[cond_name] = extract(cond, nb_top5_acc, nb_top5_label)

In [68]:
rf_base_dir = "/home/oagba/bulk/data/output_basic_15k/symptoms/csv/parsed/learning/rf"
rf_top5_file = os.path.join(rf_base_dir, "random_forest_top_5_pred.joblib")
rf_ntop5_file = os.path.join(rf_base_dir, "random_forest_top_n5_pred.joblib")

In [69]:
rf_top5_data = joblib.load(rf_top5_file)
rf_ntop5_data = joblib.load(rf_ntop5_file)

In [70]:
rf_top5_acc = rf_top5_data['acc']
rf_top5_label = rf_top5_data['labels']

In [71]:
rf_summarized = {}
for cond in eval_conditions:
    cond_name = condition_db[condition_codes[cond]]
    rf_summarized[cond_name] = extract(cond, rf_top5_acc, rf_top5_label)

In [76]:
mlp_base_dir = "/home/oagba/bulk/dl_eval/top5"
mlp_top5_file = os.path.join(mlp_base_dir, "mlp_top_5_pred.joblib")
mlp_ntop5_file = os.path.join(mlp_base_dir, "mlp_top_n5_pred.joblib")

In [77]:
mlp_top5_data = joblib.load(mlp_top5_file)
mlp_ntop5_data = joblib.load(mlp_ntop5_file)

In [78]:
mlp_top5_acc = mlp_top5_data['acc']
mlp_top5_label = mlp_top5_data['labels']

In [79]:
mlp_summarized = {}
for cond in eval_conditions:
    cond_name = condition_db[condition_codes[cond]]
    mlp_summarized[cond_name] = extract(cond, mlp_top5_acc, mlp_top5_label)

In [80]:
num_rows = len(mlp_summarized) * 5

In [91]:
headers = ["Conditions", "NB", "RF", "MLP"]
data = {item:[] for item in headers}

In [92]:
sorted_conditions = sorted(mlp_summarized.keys())

In [93]:
for cnd in sorted_conditions:
    for idx in range(5):
        data["Conditions"].append(cnd)
    
    for item in nb_summarized[cnd].keys():
        data["NB"].append(item)
    for item in rf_summarized[cnd].keys():
        data["RF"].append(item)
    for item in mlp_summarized[cnd].keys():
        data["MLP"].append(item)

In [95]:
df = pd.DataFrame(data)

In [97]:
sorted_conditions

['Acute bronchitis',
 'Acute pancreatitis',
 'Acute sinusitis',
 'Appendicitis',
 'Asthma',
 'Chronic constipation',
 'Cystitis',
 'Pyelonephritis',
 'Urethritis']

In [98]:
resp = ['Acute bronchitis', 'Acute sinusitis', 'Asthma']
gastro = ['Acute pancreatitis', 'Appendicitis', 'Chronic constipation']
uti = ['Cystitis', 'Pyelonephritis', 'Urethritis']

In [100]:
resp_df = df[df['Conditions'].isin(resp)]
gastro_df = df[df['Conditions'].isin(gastro)]
uti_df = df[df['Conditions'].isin(uti)]

In [101]:
# to_csv
op_dir = "/home/oagba/bulk/data/reports/top_5_qual"
resp_filename = os.path.join(op_dir, "resp.csv")
gastro_filename = os.path.join(op_dir, "gastro.csv")
uti_filename = os.path.join(op_dir, "uti.csv")

In [102]:
resp_df.to_csv(resp_filename, index=False)
gastro_df.to_csv(gastro_filename, index=False)
uti_df.to_csv(uti_filename, index=False)